In [ ]:
import sys  
sys.path.insert(0, '..')

import pandas as pd

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np

from util.plots import show_plot

feature = pd.read_csv('../data/feature_nirspec1000.csv').values
label = pd.read_csv('../data/label_nirspec1000.csv').values

train_feature = feature[:50000]
train_label = label[:50000]
test_feature = feature[50000:]
test_label = label[50000:]

print("Num GPUs Available: ", tf.config.list_physical_devices('GPU'))
print(train_feature.shape, train_label.shape)

## Model cu straturi dense

In [ ]:
inputs = keras.Input(shape = (1669,))

x = layers.Dense(600, activation='relu')(inputs)
x = layers.Dropout(0.2)(x)
x = layers.Dense(50, activation='relu')(x)
x = layers.Dense(50, activation='relu')(x)
x = layers.Dense(30, activation='relu')(x)

outputs = layers.Dense(12, activation='relu')(x)

model = keras.Model(inputs=inputs, outputs=outputs, name="d_model")
model.compile(
    optimizer=keras.optimizers.Nadam(),
    loss=keras.losses.MeanSquaredError(),
    metrics=['mse', 'mae']
)
model.summary()

EPOCHS = 90
history = model.fit(train_feature,
                    train_label,
                    epochs=EPOCHS, batch_size=200,
                    validation_split = 0.1)

model.save('models/d_model_1000')

print('test', model.evaluate(test_feature, test_label, batch_size=5000))
print('train', model.evaluate(train_feature, train_label, batch_size=5000))

In [ ]:
show_plot(model.predict(test_feature[:2500]), test_label[:2500],
          model.predict(train_feature[:5000]), train_label[:5000],
         (0.5, 0.5), 600)

## Modele cu straturi convoluționale

In [ ]:
train_feature_2 = train_feature[:,:,None]
test_feature_2 = test_feature[:,:,None]
print(train_feature_2.shape, test_feature_2.shape)

In [ ]:
inputs = keras.Input(shape = (1669,1))
x = layers.Conv1D(32, 3, activation='relu')(inputs)
x = layers.Flatten()(x)

y = layers.Conv1D(16, 5, activation='relu')(inputs)
y = layers.MaxPooling1D(16, 5)(y)
y = layers.Conv1D(16, 3, activation='relu')(y)
y = layers.MaxPooling1D(16, 3)(y)
y = layers.Flatten()(y)

x = layers.Dense(500, activation='relu')(layers.concatenate([x,y]))
x = layers.Dropout(0.3)(x)
x = layers.Dense(50, activation='relu')(x)

y = layers.Flatten()(inputs)
x = layers.Dense(80, activation='relu')(layers.concatenate([x, y]))
x = layers.Dropout(0.2)(x)
 
x = layers.Dense(40, activation='relu')(x)
x = layers.Dense(20, activation='relu')(x)
outputs = layers.Dense(12, activation='relu')(x)


cnn_model = keras.Model(inputs=inputs, outputs=outputs, name="cnn_model")

cnn_model.compile(
    optimizer=keras.optimizers.Nadam(),
    loss=keras.losses.MeanSquaredError(),
    metrics=['mse', 'mae']
)
cnn_model.summary()

EPOCHS = 140
history = cnn_model.fit(train_feature_2,
                        train_label,
                        epochs=EPOCHS, batch_size=200,
                        validation_split = 0.1)

cnn_model_1000 = tf.keras.models.load_model('models/cnn_model_1000')

print('test', cnn_model.evaluate(test_feature_2, test_label, batch_size=2500))
print('train', cnn_model.evaluate(train_feature_2, train_label, batch_size=2500))

In [ ]:
show_plot(cnn_model.predict(test_feature_2[:2500]), test_label[:2500],
          cnn_model.predict(train_feature_2[:5000]), train_label[:5000],
         (0.5, 0.5), 600)